In [ ]:
using System;
using System.IO;
using System.Collections.Generic;
using System.Collections.Concurrent;
using System.Collections;
using System.Threading;
using System.Text.Json;

In [ ]:
public class Debug : IDisposable
{
    public static bool IsEnabled {get; private set;} = false;

    public static void Print(string input)
    {
        if(IsEnabled)
        {
            Console.WriteLine(input);
        }
    }

    public Debug(bool enable = true)
    {
        IsEnabled = enable;
    }

    public void Dispose()
    {
        IsEnabled = false;
    }
}

In [ ]:
var input = File.ReadAllLines("input.txt");

In [ ]:
public record Unit
{
    public int Health {get; set;}
    public int Damage {get; set;}
    public int Armor {get; set;}
}

In [ ]:
var boss = new Unit();

foreach(var line in input)
{
    var x = int.Parse(line.Split(':').Last());

    if(line.Contains("Hit"))
    {
        boss.Health = x;
    }
    else if(line.Contains("Damage"))
    {
        boss.Damage = x;
    }
    else if(line.Contains("Armor"))
    {
        boss.Armor = x;
    }
}

In [ ]:
var player = new Unit
{
    Health = 100,
    Damage = 0,
    Armor = 0
};

In [ ]:
public static bool PlayerWins(Unit boss, Unit player)
{
    var give = Math.Max(player.Damage - boss.Armor, 1);
    var take = Math.Max(boss.Damage - player.Armor, 1);
    var tWin = boss.Health / give + (boss.Health % give != 0 ? 1 : 0);
    var tLos = player.Health / take + (player.Health % take != 0 ? 1 : 0);

    Debug.Print($"tWin {tWin}, tLoss {tLos}");
    
    return tLos >= tWin;
}

In [ ]:
#nullable enable

public static class Pathfinder
{
    public static PathfinderResult Dijkstra(
        BaseNode start)
    {
        long visited = 0;

        var costs = new Dictionary<int, int>();
        costs[start.Id] = start.Cost;
       
        var q = new PriorityQueue<BaseNode, int>();
        q.Enqueue(start, 0);

        while (q.TryDequeue(out var u, out var c))
        {
            visited++;

            if(visited % 10000 == 0)
            {
                Debug.Print($"Visited {visited} nodes");
            }
            
            if (u.IsGoal)
            {
                return new PathfinderResult
                {
                    Success = true,
                    GoalNode = u,
                    Visited = visited,
                    Total = c
                };
            }

            foreach (var v in u.Neighbours())
            {
                var t = c + v.Cost;

                var h = v.Id;
                if (costs.TryGetValue(h, out var cachedCost))
                {
                    if (cachedCost <= t)
                    {
                        continue;
                    }
                }

                costs[h] = t;
                
                v.Previous = u;
                q.Enqueue(v, t);
            }
        }

        return new PathfinderResult
        {
            Success = false,
            Visited = visited
        };
    }

    public static Dictionary<int, (int cost, BaseNode node)> ExhaustivePathfinder(
        BaseNode start)
    {
        long visited = 0;

        var costs = new Dictionary<int, (int cost, BaseNode node)>();
        costs[start.Id] = (start.Cost, start);
       
        var q = new PriorityQueue<BaseNode, int>();
        q.Enqueue(start, 0);

        while (q.TryDequeue(out var u, out var c))
        {
            visited++;

            if(visited % 10000 == 0)
            {
                Debug.Print($"Visited {visited} nodes");
            }
            
            foreach (var v in u.Neighbours())
            {
                var t = c + v.Cost;

                var h = v.Id;
                if (costs.TryGetValue(h, out var cache))
                {
                    if (cache.cost <= t)
                    {
                        continue;
                    }
                }

                costs[h] = (t, u);
                
                v.Previous = u;
                q.Enqueue(v, t);
            }
        }

        return costs;
    }

    public static List<BaseNode> ReconstructPath<T>(BaseNode node)
    {
        var path = new List<BaseNode>();
        BaseNode? current = node;
        while (current is not null)
        {
            path.Insert(0, current);
            current = current.Previous;
        }

        return path;
    }
}

public interface BaseNode
{
    int Cost { get; init; }
    BaseNode? Previous { get; set; }
    IEnumerable<BaseNode> Neighbours();
    bool IsGoal { get; }
    int Id { get; init; }
}

public record PathfinderResult
{
    public bool Success { get; init; }
    public BaseNode? GoalNode { get; init; }
    public long Visited { get; set; }
    public long Total {get; set;}
}

In [ ]:
public record struct Item
{
    public int Cost {get; init;}
    public int Damage {get; init;}
    public int Armor {get; init;}
    public string Name {get;init;}
}

In [ ]:
static var shop = new Dictionary<int,Item>();

var input = File.ReadAllLines("items.txt");

int i = 0;

foreach(var line in input)
{
    i++;
    var data = line.Split("  ", StringSplitOptions.RemoveEmptyEntries | StringSplitOptions.TrimEntries);
    shop.Add(i, new Item{
        Name = data[0],
        Cost = int.Parse(data[1]),
        Damage = int.Parse(data[2]),
        Armor = int.Parse(data[3])
    });
}

In [ ]:
shop.Display();

In [ ]:
static int BitCount(int n)
{
   int ret=0;
   while (n!=0)
   {
       n&=(n-1);
       ret++;
   }
   return ret;
}

In [ ]:
0b0000000000011111

In [ ]:
#nullable enable

public record GameState : BaseNode
{
    public Unit Player { get; init; }
    public Unit Boss { get; init; }
    public int Id { get; init; }
    public int Cost { get; init; }
    public BaseNode? Previous { get; set; }
    public bool IsGoal {
        get {
            if((Id & 0b0000000000011111 ) == 0){
                return false;
            }

            return PlayerWins(Boss, Player);
        }
    }

    public IEnumerable<BaseNode> Neighbours() {
        var neighbours = new List<BaseNode>();

        if((Id & 0b0000000000011111 ) == 0)
        {
            for(int i = 1; i < 6; i++){
                neighbours.Add(WithItem(i));
            }
        }

        if((Id & 0b0000001111100000 ) == 0)
        {
            for(int i = 6; i < 11; i++){
                neighbours.Add(WithItem(i));
            }
        }

        if(BitCount(Id & 0b1111110000000000) < 2)
        {
            for(int i = 11; i < 17; i++){
                if((Id & 0b1 << (i - 1)) == 0)
                    neighbours.Add(WithItem(i));
            }
        }

        return neighbours;
    }

    public string Items {get; init;} = "";

    public BaseNode WithItem(int item) => this with{
        Id = Id + (0b1 << (item - 1)),
        Player = Player with {
            Armor = Player.Armor + shop[item].Armor,
            Damage = Player.Damage + shop[item].Damage
        },
        Cost = shop[item].Cost,
        Items = Items + shop[item].Name + " "
    };
    
}

In [ ]:
var goal = Pathfinder.Dijkstra(new GameState{
    Boss = boss,
    Player = player
}).Display();


In [ ]:
using(new Debug())
{
    PlayerWins(new Unit{Health= 109, Armor = 2, Damage = 8}, new Unit{Health= 100, Armor = 3, Damage = 7}).Display();
}

In [ ]:
var goal = Pathfinder.ExhaustivePathfinder(new GameState{
    Boss = boss,
    Player = player
}).Where(c => !c.Value.node.IsGoal).OrderByDescending(c => c.Value.cost).Select(c => (c.Value.node as GameState).Items).Display();